In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import math
from torch.utils.data import Dataset
import pickle
import datetime
import pandas as pd


In [6]:
torch.cuda.is_available()

True

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import csv
training_data = []
with open('/content/drive/MyDrive/database_paris_marseille copy.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        row[0] = float(row[0])
        if row[0]>90:
            row[0] = row[0]-360
        training_data.append([float(item) for item in row])

with open("/content/drive/MyDrive/objet_wind_data_2020.pickle", "rb") as f:
    wind_data = pickle.load(f)
    data = wind_data['data']



In [26]:
data_std = 1

def grid_time(t):
    t = t
    t = t/3600
    t = t + 1 + 4
    t = t - t%6
    t = t - 1
    return (t+1)/6

def grid_lat(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5
    l = l/2
    l = l +90
    l = l*2

    return (l/5)%73

def grid_long(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5

    return (l/5)%144
def int_array(arr):
    return [int(i) for i in arr]

def grid_array(arr):
    return int_array([grid_long(arr[0]),grid_lat(arr[1]),grid_time(arr[2]),int(arr[3]),int(arr[4])])

def get_winds_at_t(data,time,dtime):
    tensor = torch.tensor([data[time:time+dtime,:,:,:]], dtype=torch.float32).permute(1,0,5,2,3,4)
    return tensor

def input_label(training_data,dtime,i, goal_coord,data):
    array = grid_array(training_data[i])
    time = array[2]
    long = array[0]
    lat = array[1]
    alt = array[3]
    action = array[4]
    #winds = torch.tensor([data[time:time+dtime,:,:,:]], dtype=torch.float32).permute(1,0,5,2,3,4)
    winds = data[time:time+dtime,:,:,:]
    winds = np.transpose(winds,(0,4,1,2,3))
    winds = np.arctan2(winds[:,0:1,:,:,:],winds[:,1:2,:,:,:])/(3.1415/2)
    pos = np.zeros((dtime,1,winds.shape[2],winds.shape[3],winds.shape[4]))
    for i in range(pos.shape[2]):
        pos[0,0,i,int(grid_long(goal_coord[0])),int(grid_lat(goal_coord[1]))] = -(3)
    pos[0,0,alt,long,lat] = 3
    input = np.concatenate((winds,pos),axis=1)

    return (input,action)


In [10]:
coords_destination = [5.3698, 43.2965]

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv3d(2, 32, 5, padding=(2,2,2))
        self.conv2 = nn.Conv3d(32, 32, 3, padding=(2,2,2))
        self.conv3 = nn.Conv3d(32, 64, 3, padding=(1,1,1))
        self.conv4 = nn.Conv3d(64, 128, 3, padding=(1,1,1))
        self.conv5 = nn.Conv3d(128, 256, 3, padding=(1,1,1))
        self.conv6 = nn.Conv3d(256, 256, 3, padding=(1,1,1))
        self.conv7 = nn.Conv3d(256, 384, 3, padding=(1,1,1))

        self.fc1 = nn.Linear(384, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 3)
        self.drop = nn.Dropout(0.3)

        self.batch1 = nn.BatchNorm3d(32)
        self.batch2 = nn.BatchNorm3d(32)
        self.batch3 = nn.BatchNorm3d(64)
        self.batch4 = nn.BatchNorm3d(128)
        self.batch5 = nn.BatchNorm3d(256)
        self.batch6 = nn.BatchNorm3d(256)
        self.batch7 = nn.BatchNorm3d(384)

        self.batch8 = nn.BatchNorm1d(512)
        self.batch9 = nn.BatchNorm1d(512)
        self.batch10 = nn.BatchNorm1d(512)


        self.pool= nn.MaxPool3d(kernel_size=(1,3,3),stride = (1,2,2))
        self.pool1= nn.MaxPool3d(kernel_size=(2,2,2),stride = (2,2,2))
        self.pool2= nn.MaxPool3d(kernel_size=(1,2,1),stride = (1,2,1))

        # self.pool1 = nn.MaxPool3d(kernel_size=,stride = 2)


        n = self.conv1.kernel_size[0] * self.conv1.kernel_size[1] * self.conv1.kernel_size[2]* self.conv1.out_channels
        self.conv1.weight.data.normal_(0, math.sqrt(2. / n))
        n = self.conv2.kernel_size[0] * self.conv2.kernel_size[1] * self.conv2.kernel_size[2]* self.conv2.out_channels
        self.conv2.weight.data.normal_(0, math.sqrt(2. / n))
        n = self.conv3.kernel_size[0] * self.conv3.kernel_size[1] * self.conv3.kernel_size[2]* self.conv3.out_channels
        self.conv3.weight.data.normal_(0, math.sqrt(2. / n))
        n = self.conv4.kernel_size[0] * self.conv4.kernel_size[1] * self.conv4.kernel_size[2]* self.conv4.out_channels
        self.conv4.weight.data.normal_(0, math.sqrt(2. / n))
        n = self.conv5.kernel_size[0] * self.conv5.kernel_size[1] * self.conv5.kernel_size[2]* self.conv5.out_channels
        self.conv5.weight.data.normal_(0, math.sqrt(2. / n))
        n = self.conv6.kernel_size[0] * self.conv6.kernel_size[1] * self.conv6.kernel_size[2]* self.conv6.out_channels
        self.conv6.weight.data.normal_(0, math.sqrt(2. / n))
        n = self.conv7.kernel_size[0] * self.conv7.kernel_size[1] * self.conv7.kernel_size[2]* self.conv7.out_channels
        self.conv7.weight.data.normal_(0, math.sqrt(2. / n))


        nn.init.xavier_uniform(self.fc1.weight)
        nn.init.xavier_uniform(self.fc2.weight)
        nn.init.xavier_uniform(self.fc3.weight)
        nn.init.xavier_uniform(self.fc4.weight)

        self.batch1.weight.data.fill_(1)
        self.batch1.bias.data.zero_()
        self.batch2.weight.data.fill_(1)
        self.batch2.bias.data.zero_()
        self.batch3.weight.data.fill_(1)
        self.batch3.bias.data.zero_()
        self.batch5.weight.data.fill_(1)
        self.batch5.bias.data.zero_()
        self.batch4.weight.data.fill_(1)
        self.batch4.bias.data.zero_()
        self.batch6.weight.data.fill_(1)
        self.batch6.bias.data.zero_()
        self.batch7.weight.data.fill_(1)
        self.batch7.bias.data.zero_()
        self.batch8.weight.data.fill_(1)
        self.batch8.bias.data.zero_()
        self.batch9.weight.data.fill_(1)
        self.batch9.bias.data.zero_()
        self.batch10.weight.data.fill_(1)
        self.batch10.bias.data.zero_()


    def forward(self, x):
        flats = []
        x = torch.tanh(x/data_std)
        _,n,_,_,_,_ = x.shape
        for i in range(n):
            y = x[:,i,:,:,:,:]
            #print(y.shape)
            y = F.relu(self.batch1(self.conv1(y)))
            y = self.pool(y)
            y = F.relu(self.batch2(self.conv2(y)))
            y = self.pool2(y)
            y = F.relu(self.batch3(self.conv3(y)))
            y = self.pool(y)
            y = F.relu(self.batch4(self.conv4(y)))
            y = self.pool1(y)
            y = F.relu(self.batch5(self.conv5(y)))
            y = self.pool1(y)
            y = F.relu(self.batch6(self.conv6(y)))
            y = self.pool1(y)
            y = F.relu(self.batch7(self.conv7(y)))
            y = self.pool1(y)
            y = torch.flatten(y, 1)
            flats.append(y)
        x = torch.cat(flats, dim=1)
        x = self.drop(x)
        x = F.relu(self.batch8(self.fc1(x)))
        x = self.drop(x)
        x = F.relu(self.batch9(self.fc2(x)))
        x = self.drop(x)
        x = F.relu(self.batch10(self.fc3(x)))
        x = self.drop(x)
        x = self.fc4(x)
        #x = F.softmax(x, dim=1)
        return x

In [30]:
import os
import pandas as pd

class CustomDataset(Dataset):
    def __init__(self, entries, dtime, goal_coord, data, transform=None, target_transform=None):
        self.entries = entries
        self.transform = transform
        self.target_transform = target_transform
        self.dtime = dtime
        self.goal_coord = goal_coord
        self.data = data


    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        inp,lab = input_label(self.entries,self.dtime,idx, self.goal_coord,self.data)
        print(inp.shape)
        inp = np.pad(inp,((0,0),(0,0),(0,0),(2,2),(0,0)),'wrap')
        return inp, lab

In [31]:
trainset = CustomDataset(training_data,1,coords_destination,data)
batch_size = 64
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)


In [13]:
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
print(dev)

cuda:0


In [14]:
net = Net()
net = net.to(dev)


NameError: name 'Net' is not defined

In [16]:
#criterion = nn.MultiMarginLoss()à
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.00001)


NameError: name 'net' is not defined

In [32]:
a,b = next(iter(trainloader))
a = a.float().to(dev)
print(a.shape)
r = net(a)


(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)
(1, 2, 17, 144, 73)


NameError: name 'net' is not defined

In [ ]:
for epoch in range(50):  # loop over the dataset multiple times
    running_loss = 0.0
    # exp_lr_scheduler.step()
    for i, batch in enumerate(trainloader, 0):
        inputs, labels = batch
        inputs = inputs.float().to(dev)
        labels = labels.to(dev)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 99))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 1.536
[1,   200] loss: 1.454
[1,   300] loss: 1.438
[1,   400] loss: 1.408
[1,   500] loss: 1.406


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images
        labels = labels
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

NameError: name 'testloader' is not defined